In [1]:
import torch
import math

# Just use for the plots at the end
import matplotlib.pyplot as plt
from numpy import exp,arange
import numpy as np
import math
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show

torch.set_grad_enabled(False)

In [2]:
class Module(object) :
    def __init__(self):
        super().__init__()
    
    def forward(self , *input):
        raise  NotImplementedError
        
    def backward(self , *gradwrtoutput):
        raise  NotImplementedError
        
    def param(self): # These are the layers of the network
        return  []

In [3]:
class Layer(Module):
    def __init__(self):
        super().__init__()
        self.dropout = False
        self.linear = False
        
    def is_dropout(self):
        self.dropout = True
    
    def is_linear(self):
        self.linear = True

In [4]:
class Sequential(Module):
    def __init__(self, param, Loss):
        super().__init__()
        self.model = (param)
        self.loss = Loss
    
    def forward(self, x):
        for layer in self.model:
            x = layer.forward(x)
        return x
    
    def backward(self, output, target):
        grad = self.loss.backward(target, output)
        
        for layer in reversed(self.model):
            grad = layer.backward(grad)
        
        Loss = self.loss.forward(target, output)
        return Loss
    
    def Train(self):
        for layer in self.model:
            if layer.dropout:
                layer.Train()
        
    def Eval(self):
        for layer in self.model:
            if layer.dropout:
                layer.Eval()
    
    def lr_method(self, method, lr):
        for layer in self.model:
            if layer.linear:
                layer.change_lr_method(method, lr)

In [5]:
class Linear(Layer):
    def __init__(self, in_, out_):
        super().__init__()
        self.in_ = in_
        self.out_ = out_
        self.is_linear()
        self.lr = 0.005
        self.lr_method = 'constant'
        
        # Capture the term at each layer before the passage in the layer
        # and the activation function.
        self.x = torch.zeros(out_)
        
        # Initialization of Adam for weight and bias
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.eps = 1.0e-8
        self.eta = 1.0e-1
        self.mw = torch.zeros(out_)
        self.mb = torch.zeros(out_)
        self.vw = 0.0
        self.vb = 0.0
        
        # Initialization of the weights and the bias
        param = 1. / math.sqrt(in_)
        self.weight = torch.empty(self.in_, self.out_).uniform_(-param, param)
        self.bias = torch.empty(self.out_).uniform_(-param, param)
        
    def forward(self, x):
        self.x = x
        return x.mm(self.weight) + self.bias
    
    def set_Lr(self, lr):
        self.lr = lr
        return
        
    def backward(self, grad):
        
        if self.lr_method == "Adam":
            
            # Adam method for the learning rate
            gw = self.x.t().mm(grad)
            self.mw = ((self.beta1 * self.mw) + ((1 - self.beta1) * gw))
            mh = (1 / (1 - self.beta1)) * self.mw
            self.vw = ((self.beta2 * self.vw) + ((1 - self.beta2) * (gw.norm()**2)))
            vh = (1 / (1 - self.beta2)) * self.vw
            self.weight = self.weight - ((self.eta / (vh.sqrt() + self.eps)) * mh)

            self.mb = ((self.beta1 * self.mb) + ((1 - self.beta1) * grad))
            mh = (1 / (1 - self.beta1)) * self.mb
            self.vb = ((self.beta2 * self.vb) + ((1 - self.beta2) * (grad.norm()**2)))
            vh = (1 / (1 - self.beta2)) * self.vb
            self.bias = self.bias - ((self.eta / (vh.sqrt() + self.eps)) * mh)
            grad = grad.mm(self.weight.t())
            
        elif self.lr_method == "constant":
            
            # Constant learning rate
            self.weight = self.weight - self.lr * self.x.t().mm(grad)
            self.bias = self.bias - self.lr * grad * 1
            grad = grad.mm(self.weight.t())
            
        return grad
    
    def weight(self):
        return self.weight
    
    def bias(self):
        return self.bias
    
    def change_lr_method(self, method, lr):
        self.lr = lr
        self.lr_method = method

In [6]:
class Dropout(Layer) :
    def __init__(self):
        super().__init__()
        self.p = 0.
        self.is_dropout()
        self.train = True
        
    
    def forward(self, x):
        n = torch.ones(x.size())
        if self.train:
            n = torch.bernoulli(n) * (1 - self.p)
        return x * n
        
    def backward(self, x):
        return x
    
    def Train(self):
        self.train = True
        
    def Eval(self):
        self.train = False
        

In [7]:
class LossMSE(Layer):
    def __init__(self):
        super().__init__() 
    
    def forward(self, data_target, data_output):
        loss = (data_output - data_target).pow(2).sum()
        return loss
    
    def backward(self, data_target, data_output):
        dloss = 2 * (data_output - data_target)
        return dloss
    
    def is_MSE(self):
        return True

In [8]:
class CrossEntropyLoss(Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, data_target, data_output):
        output = data_output.to(dtype=torch.float)
        target = data_target.resize_(data_target.size(0), 1)
        
        zer = torch.zeros(target.size()).int()
        target = torch.cat((target,zer), 1)
    
        first_column = torch.tensor([0])
        loss = output.gather(1,target).index_select(1,first_column).exp()
        
        # To avoid numerical error in the computation
        maxx = loss.max()
        
        loss = (loss * maxx) / (output.exp().sum(1) * maxx)
        loss = -(loss.log().mean())
        return loss
    
    def backward(self, data_target, data_output):
        # New version
        N = data_target.size(0)
        dloss = data_output.exp()
        dloss = dloss / dloss.sum(1).resize_(N,1)
        
        add = data_target-1
        add = torch.cat((add, -data_target), 1)
        dloss = (1/N) * (dloss + add)
        return dloss
    
    def is_MSE(self):
        return False

In [9]:
class ReLU(Layer):
    
    def __init__(self ):
        super().__init__()
        self.save = 0
        
    def forward(self, x):
        y = x.clamp(min = 0)
        self.save = x
        return y
    
    def backward(self, x):
        y = self.save > 0
        return y.float() * x
         
    def print(self):
        return

In [36]:
class Leaky_ReLU(Layer):
    
    def __init__(self ):
        super().__init__()
        self.s = 0
        self.alpha = 0.01
        
    def forward(self, x):
        y = torch.maximum(self.alpha * x, x)
        self.s = x
        return y
    
    def backward(self, x):
        y = ((self.s > 0) * (1 - self.alpha)) + self.alpha
        return y.float() * x
         
    def print(self):
        return

In [34]:
class ELU(Layer):
    
    def __init__(self):
        super().__init__()
        self.s = 0
        self.alpha = 0.01
        
    def forward(self, x):
        y = ((x > 0).float() * x) + (0 >= x) * self.alpha * (torch.exp(x) - 1)
        self.s = x
        return y
    
    def backward(self, x):
        y = ((self.s > 0) * (1 - self.alpha * torch.exp(self.s))) + self.alpha * torch.exp(self.s)
        return y.float() * x

In [12]:
class Tanh(Layer) :
    def __init__(self, ):
        super().__init__()
        self.save = 0
    
    def  forward(self, x):
        self.save = x
        return torch.div(x.exp() - (-x).exp(), x.exp() + (-x).exp())
        
    def  backward(self, x):
        return (1 - torch.div(self.save.exp() - 
                    (-self.save).exp(), self.save.exp() + (-self.save).exp())**2) * x
        
    def print(self):
        return

In [13]:
class Sigmoid(Layer):
    
    def __init__(self):
        super().__init__()
        self.s = 0
        self.lbd = 3
        
    def forward(self, x):
        y = 1 / (1 + torch.exp(-self.lbd * x))
        self.s = x
        return y
    
    def backward(self, x):
        y = self.lbd * torch.exp(-self.s) / ((1 + torch.exp(-self.lbd * self.s))**2)
        return y.float() * x  

In [14]:
def create_random_batch(input_size, mini_batch_size):
    
    # This function return a 2D tensor that is the rando selection of inputs for our
    # stochastic gradient method, taking in count the number of mini_batches.
    
    # We suppose here that our mini_batch_size is well chosen taking in count the fact
    # that it divides input_size.
    
    # Initialization
    L = int(input_size / mini_batch_size)
    new_batch = torch.ones(L, mini_batch_size)
    
    indices = torch.randperm(input_size)
    for k in range(L):
        new_batch[k] = indices[k * mini_batch_size : (k+1) * mini_batch_size]
    
    return new_batch

In [15]:
def train_model(model, train_input, train_classes, nb_epochs, mini_batch_size):
    
    for epoch in range(nb_epochs):
        random_batches = create_random_batch(train_input.size(0), mini_batch_size).long()
        for batch in random_batches:
            output = model.forward(train_input[batch])
            loss = model.backward(output, train_classes[batch])

In [16]:
def compute_nb_errors(model, data_input, data_target, mini_batch_size):
    
    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        result = model.forward(data_input.narrow(0, b, mini_batch_size))
        
        # Display which allows the diagnosis of the dying-ReLU: 
        # result is always equal to 0 when ReLU is not working.
        # print(result)
        
        if model.loss.is_MSE():
            # If the loss function is MSE
            predicted_classes = (result >= 0.5).int()
        else:
            # If the loss function is CrossEntropy
            _, predicted_classes = torch.max(result, 1)
        
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1
                
    return nb_data_errors

In [17]:
def train_and_test_model(Models, train_input, train_classes, Tests, nb_epochs, mini_batch_size):
    
    Train_error = []
    Test_error = []
    std_deviation = 0.0
    train_error = 0.0
    avg_nb_test_error = torch.tensor(())
    
    for k in range (0, len(Models)):
        model = Models[k]
        
        model.lr_method("Adam", 1.0e-3)
        
        train_model(model, train_input, train_classes, nb_epochs, mini_batch_size)
        
        model.Eval()
        nb_train_errors = compute_nb_errors(model, train_input, train_classes, mini_batch_size)
        train_error += nb_train_errors / 10
            
        nb_test_errors = compute_nb_errors(model, Tests[k][0], Tests[k][1], mini_batch_size)
        model.Train()
        nb_test_errors = torch.tensor([nb_test_errors / 10]).float()
        avg_nb_test_error = torch.cat((avg_nb_test_error, nb_test_errors), 0)
        
    Train_error.append(train_error / len(Models))
    Test_error.append(avg_nb_test_error.mean().tolist())
    std_deviation = avg_nb_test_error.std().tolist()
    
    return Train_error, Test_error, std_deviation

In [18]:
def create_problem(nb_samples):
    
    # Remark: the function .uniform return a uniform distribution on [0,1) instead of [0,1],
    # but in our case it's not a problem since it is only a train and a test set on a circle
    # that do not touch the border of the set [0,1]^2.
    train_input = torch.empty(nb_samples, 2).uniform_(0, 1)
    test_input = torch.empty(nb_samples, 2).uniform_(0, 1)
    
    # Radius of our circle
    R = 1 / math.sqrt(2 * math.pi)
    
    train_classes = train_input.sub(0.5).pow(2).sum(1).sub(R**2).sign().sub(1).div(-2).long().resize_((nb_samples,1))
    test_classes = test_input.sub(0.5).pow(2).sum(1).sub(R**2).sign().sub(1).div(-2).long().resize_((nb_samples,1))
    
    return train_input, train_classes, test_input, test_classes

In [19]:
def get_tests(n):
    M = []
    for k in range (n):
        L = []
        _, _, test_input, test_classes =  create_problem(1000)
        L.append(test_input)
        L.append(test_classes)
        M.append(L)
    return M

In [20]:
def soft_max(data_output):
    output = data_output.to(dtype=torch.float)
    L = output.exp()
    
    # To avoid numerical error in the computation
    maxx = L.max()
    
    L = torch.div((L * maxx), (output.exp().sum(1).resize(L.size(0),1) * maxx))
    return L

In [21]:
nb_epochs = 100
mini_batch_size = 10
nb_rounds = 10
train_input, train_classes, _, _ = create_problem(1000)
Tests = get_tests(nb_rounds)

In [22]:
# TU COMMENCES ICI

In [23]:
# Initialization of our different architectures

def get_Models(Loss, nb_rounds):
    
    Model_ReLU_List = []
    Model_Tanh_List = []
    Model_Sigmoid_List = []
    Model_Leaky_ReLU_List = []
    Model_ELU_List = []
    
    out = 2
    if (Loss.is_MSE()):
        out = 1
    
    for k in range (0, nb_rounds):
        Model_ReLU_List.append(Sequential([Linear(2,25), ReLU(), Linear(25,25), ReLU(), Linear(25,25), ReLU(), Linear(25,out), ReLU()], Loss))
        Model_Tanh_List.append(Sequential([Linear(2,25), Tanh(), Linear(25,25), Tanh(), Linear(25,25), Tanh(), Linear(25,out), Tanh()], Loss))
        Model_Sigmoid_List.append(Sequential([Linear(2,25), Sigmoid(), Linear(25,25), Sigmoid(), Linear(25,25), Sigmoid(), Linear(25,out), Sigmoid()], Loss))
        Model_Leaky_ReLU_List.append(Sequential([Linear(2,25), Leaky_ReLU(), Linear(25,25), Leaky_ReLU(), Linear(25,25), Leaky_ReLU(), Linear(25,out), Leaky_ReLU()], Loss))
        Model_ELU_List.append(Sequential([Linear(2,25), ELU(), Linear(25,25), ELU(), Linear(25,25), ELU(), Linear(25,out), ELU()], Loss))
        
    return Model_ReLU_List, Model_Tanh_List, Model_Sigmoid_List, Model_Leaky_ReLU_List, Model_ELU_List

In [24]:
# Loss = MSE Loss

In [25]:
Model_ReLU_MSE, Model_Tanh_MSE, Model_Sigmoid_MSE, Model_Leaky_ReLU_MSE, Model_ELU_MSE = \
    get_Models(LossMSE(), nb_rounds)

In [26]:
Train_error_ReLU_MSE, Test_error_ReLU_MSE, std_deviation_ReLU_MSE = \
    train_and_test_model(Model_ReLU_MSE, train_input, train_classes, Tests, nb_epochs, mini_batch_size)

#Average train_error (%) on number_of_rounds Sequential models with ReLU and LossMSE 
print("Average train_error on", nb_rounds,"Sequential models with ReLU and LossMSE is",Train_error_ReLU_MSE[0],'%')

#Average test_error (%) on number_of_rounds Sequential models with ReLU and LossMSE 
print("Average test_error on", nb_rounds,"Sequential models with ReLU and LossMSE is",Test_error_ReLU_MSE[0],"%")

#Standard deviation corresponding to the average on test_error just above
print("Standard deviation corresponding to the average on test_error just above is ", std_deviation_ReLU_MSE, "%")

Average train_error on 10 Sequential models with ReLU and LossMSE is 20.87 %
Average test_error on 10 Sequential models with ReLU and LossMSE is 21.920001983642578 %
Standard deviation corresponding to the average on test_error just above is  23.53195571899414 %


In [27]:
Train_error_Tanh_MSE, Test_error_Tanh_MSE, std_deviation_Tanh_MSE = \
    train_and_test_model(Model_Tanh_MSE, train_input, train_classes, Tests, nb_epochs, mini_batch_size)

#Average train_error (%) on number_of_rounds Sequential models with Tanh and LossMSE
print("Average train_error on", nb_rounds,"Sequential models with Tanh and LossMSE is",Train_error_Tanh_MSE[0],'%')

#Average test_error (%) on number_of_rounds Sequential models with Tanh and LossMSE 
print("Average test_error on", nb_rounds,"Sequential models with Tanh and LossMSE is",Test_error_Tanh_MSE[0],"%")

#Standard deviation corresponding to the average on test_error just above
print("Standard deviation corresponding to the average on test_error just above is ", std_deviation_Tanh_MSE, "%")

Average train_error on 10 Sequential models with Tanh and LossMSE is 4.64 %
Average test_error on 10 Sequential models with Tanh and LossMSE is 4.970000267028809 %
Standard deviation corresponding to the average on test_error just above is  1.9102355241775513 %


In [28]:
Train_error_Sigmoid_MSE, Test_error_Sigmoid_MSE, std_deviation_Sigmoid_MSE = \
    train_and_test_model(Model_Sigmoid_MSE, train_input, train_classes, Tests, nb_epochs, mini_batch_size)

#Average train_error (%) on number_of_rounds Sequential models with Sigmoid and LossMSE 
print("Average train_error on", nb_rounds,"Sequential models with Sigmoid and LossMSE is",Train_error_Sigmoid_MSE[0],'%')

#Average test_error (%) on number_of_rounds Sequential models with Sigmoid and LossMSE 
print("Average test_error on", nb_rounds,"Sequential models with Sigmoid and LossMSE is",Test_error_Sigmoid_MSE[0],"%")

#Standard deviation corresponding to the average on test_error just above
print("Standard deviation corresponding to the average on test_error just above is ", std_deviation_Sigmoid_MSE, "%")

Average train_error on 10 Sequential models with Sigmoid and LossMSE is 13.319999999999999 %
Average test_error on 10 Sequential models with Sigmoid and LossMSE is 12.649999618530273 %
Standard deviation corresponding to the average on test_error just above is  12.538938522338867 %


In [29]:
Train_error_Leaky_ReLU_MSE, Test_error_Leaky_ReLU_MSE, std_deviation_Leaky_ReLU_MSE = \
    train_and_test_model(Model_Leaky_ReLU_MSE, train_input, train_classes, Tests, nb_epochs, mini_batch_size)

#Average train_error (%) on number_of_rounds Sequential models with Leaky_ReLU and LossMSE 
print("Average train_error on", nb_rounds,"Sequential models with Leaky_ReLU and LossMSE is",Train_error_Leaky_ReLU_MSE[0],'%')

#Average test_error (%) on number_of_rounds Sequential models with Leaky_ReLU and LossMSE 
print("Average test_error on", nb_rounds,"Sequential models with Leaky_ReLU and LossMSE is",Test_error_Leaky_ReLU_MSE[0],"%")

#Standard deviation corresponding to the average on test_error just above
print("Standard deviation corresponding to the average on test_error just above is ", std_deviation_Leaky_ReLU_MSE, "%")

Average train_error on 10 Sequential models with Leaky_ReLU and LossMSE is 3.9299999999999997 %
Average test_error on 10 Sequential models with Leaky_ReLU and LossMSE is 4.309999465942383 %
Standard deviation corresponding to the average on test_error just above is  1.3827911615371704 %


In [30]:
Train_error_ELU_MSE, Test_error_ELU_MSE, std_deviation_ELU_MSE = \
    train_and_test_model(Model_ELU_MSE, train_input, train_classes, Tests, nb_epochs, mini_batch_size)


#Average train_error (%) on number_of_rounds Sequential models with ELU and LossMSE 
print("Average train_error on", nb_rounds,"Sequential models with ELU and LossMSE is",Train_error_ELU_MSE[0],'%')

#Average test_error (%) on number_of_rounds Sequential models with ELU and LossMSE 
print("Average test_error on", nb_rounds,"Sequential models with ELU and LossMSE is",Test_error_ELU_MSE[0],"%")

#Standard deviation corresponding to the average on test_error just above
print("Standard deviation corresponding to the average on test_error just above is ", std_deviation_ELU_MSE, "%")

Average train_error on 10 Sequential models with ELU and LossMSE is 3.4799999999999995 %
Average test_error on 10 Sequential models with ELU and LossMSE is 3.8400001525878906 %
Standard deviation corresponding to the average on test_error just above is  0.8579044342041016 %


In [31]:
# Loss = Cross-Entropy Loss

In [32]:
Model_ReLU_CE, Model_Tanh_CE, Model_Sigmoid_CE, Model_Leaky_ReLU_CE, Model_ELU_CE = \
    get_Models(CrossEntropyLoss(), nb_rounds)

In [33]:
Train_error_Tanh_CE, Test_error_Tanh_CE, std_deviation_Tanh_CE = \
    train_and_test_model(Model_Tanh_CE, train_input, train_classes, Tests, nb_epochs, mini_batch_size)

#Average train_error (%) on number_of_rounds Sequential models with Tanh and Cross-Entropy Loss
print("Average train_error on", nb_rounds,"Sequential models with Tanh and Cross-Entropy Loss is",Train_error_Tanh_CE[0],'%')

#Average test_error (%) on number_of_rounds Sequential models with Tanh and Cross-Entropy Loss
print("Average test_error on", nb_rounds,"Sequential models with Tanh and Cross-Entropy Loss is",Test_error_Tanh_CE[0],"%")

#Standard deviation corresponding to the average on test_error just above
print("Standard deviation corresponding to the average on test_error just above is ", std_deviation_Tanh_CE, "%")

Average train_error on 10 Sequential models with Tanh and Cross-Entropy Loss is 8.370000000000001 %
Average test_error on 10 Sequential models with Tanh and Cross-Entropy Loss is 9.149999618530273 %
Standard deviation corresponding to the average on test_error just above is  15.187366485595703 %


In [34]:
Train_error_Sigmoid_CE, Test_error_Sigmoid_CE, std_deviation_Sigmoid_CE = \
    train_and_test_model(Model_Sigmoid_CE, train_input, train_classes, Tests, nb_epochs, mini_batch_size)

#Average train_error (%) on number_of_rounds Sequential models with Sigmoid and Cross-Entropy Loss
print("Average train_error on", nb_rounds,"Sequential models with Sigmoid and Cross-Entropy Loss is",Train_error_Sigmoid_CE[0],'%')

#Average test_error (%) on number_of_rounds Sequential models with Sigmoid and Cross-Entropy Loss
print("Average test_error on", nb_rounds,"Sequential models with Sigmoid and Cross-Entropy Loss is",Test_error_Sigmoid_CE[0],"%")

#Standard deviation corresponding to the average on test_error just above
print("Standard deviation corresponding to the average on test_error just above is ", std_deviation_Sigmoid_CE, "%")

Average train_error on 10 Sequential models with Sigmoid and Cross-Entropy Loss is 51.4 %
Average test_error on 10 Sequential models with Sigmoid and Cross-Entropy Loss is 50.27000045776367 %
Standard deviation corresponding to the average on test_error just above is  1.9510960578918457 %


In [35]:
Train_error_Leaky_ReLU_CE, Test_error_Leaky_ReLU_CE, std_deviation_Leaky_ReLU_CE = \
    train_and_test_model(Model_Leaky_ReLU_CE, train_input, train_classes, Tests, nb_epochs, mini_batch_size)

#Average train_error (%) on number_of_rounds Sequential models with Leaky_ReLU and Cross-Entropy Loss
print("Average train_error on", nb_rounds,"Sequential models with Leaky_ReLU and Cross-Entropy Loss is",Train_error_Leaky_ReLU_CE[0],'%')

#Average test_error (%) on number_of_rounds Sequential models with Leaky_ReLU and Cross-Entropy Loss 
print("Average test_error on", nb_rounds,"Sequential models with Leaky_ReLU and Cross-Entropy Loss is",Test_error_Leaky_ReLU_CE[0],"%")

#Standard deviation corresponding to the average on test_error just above
print("Standard deviation corresponding to the average on test_error just above is ", std_deviation_Leaky_ReLU_CE, "%")

Average train_error on 10 Sequential models with Leaky_ReLU and Cross-Entropy Loss is 25.21 %
Average test_error on 10 Sequential models with Leaky_ReLU and Cross-Entropy Loss is 26.560001373291016 %
Standard deviation corresponding to the average on test_error just above is  23.983291625976562 %


In [36]:
Train_error_ELU_CE, Test_error_ELU_CE, std_deviation_ELU_CE = \
    train_and_test_model(Model_ELU_CE, train_input, train_classes, Tests, nb_epochs, mini_batch_size)


#Average train_error (%) on number_of_rounds Sequential models with ELU and Cross-Entropy Loss
print("Average train_error on", nb_rounds,"Sequential models with ELU and Cross-Entropy Loss is",Train_error_ELU_CE[0],'%')

#Average test_error (%) on number_of_rounds Sequential models with ELU and Cross-Entropy Loss
print("Average test_error on", nb_rounds,"Sequential models with ELU and Cross-Entropy Loss is",Test_error_ELU_CE[0],"%")

#Standard deviation corresponding to the average on test_error just above
print("Standard deviation corresponding to the average on test_error just above is ", std_deviation_ELU_CE, "%")

Average train_error on 10 Sequential models with ELU and Cross-Entropy Loss is 29.919999999999998 %
Average test_error on 10 Sequential models with ELU and Cross-Entropy Loss is 30.299999237060547 %
Standard deviation corresponding to the average on test_error just above is  21.708011627197266 %


In [ ]:
#Dropout

In [ ]:
Model_Tanh_Dropout 

In [47]:
# PAS TOUCHE

train_input, train_classes, _, _ = create_problem(1000)
#print(train_input.size())
#print(train_input.narrow(0, b, mini_batch_size).size())
nb_epochs = 100
mini_batch_size = 1

model = Sequential([Linear(2,25), Tanh(), Linear(25,25), Tanh(), Linear(25,25), Tanh(), Linear(25,2), Tanh()], CrossEntropyLoss())
model.lr_method("Adam", 1.0e-3)
train_model(model, train_input, train_classes, nb_epochs, mini_batch_size)

In [48]:
nb_train_errors = compute_nb_errors(model, train_input, train_classes, mini_batch_size)
print('train error {:0.2f}% {:f}/{:f}'.format((100 * nb_train_errors) / train_input.size(0), nb_train_errors, train_classes.size(0)))

L = get_tests(10)
average_nb_test_error = 0
for k in range (0, len(L)):
    nb_test_errors = compute_nb_errors(model, L[k][0], L[k][1], mini_batch_size)
    average_nb_test_error += nb_test_errors
    print('test error {:0.2f}% {:f}/{:f}'.format((100 * nb_test_errors) / L[k][0].size(0), nb_test_errors, L[k][0].size(0)))
print('Average test error {:0.2f}% {:0.1f}/{:d}'.format((100*average_nb_test_error/len(L)) / L[0][0].size(0), average_nb_test_error/len(L), L[0][0].size(0)))

train error 50.90% 509.000000/1000.000000
test error 50.80% 508.000000/1000.000000
test error 50.20% 502.000000/1000.000000
test error 50.70% 507.000000/1000.000000
test error 48.50% 485.000000/1000.000000
test error 49.10% 491.000000/1000.000000
test error 49.20% 492.000000/1000.000000
test error 47.60% 476.000000/1000.000000
test error 46.40% 464.000000/1000.000000
test error 49.90% 499.000000/1000.000000
test error 51.10% 511.000000/1000.000000
Average test error 49.35% 493.5/1000


In [49]:
x = torch.linspace(0.0, 1.0, 1000)
y = torch.linspace(0.0, 1.0, 1000)
n = x.size(0)
grid_x, grid_y = torch.meshgrid(x,y)
Z = torch.empty(grid_x.size())
for k in range(grid_x.size(0)):
    for l in range(0, n, mini_batch_size):        
        xx = grid_x[k][l:l+mini_batch_size].t().resize_((mini_batch_size,1))
        yy = grid_y[k][l:l+mini_batch_size].t().resize_((mini_batch_size,1))
        e = torch.cat([xx, yy], 1)
        
        if model.loss.is_MSE():
            # MSE Loss
            Z[k][l:l+mini_batch_size] = model.forward(e).resize(mini_batch_size)
        else:
            # Cross Entropy loss
            output = model.forward(e)
            output = soft_max(output)

            second_column = torch.tensor([1])
            Z[k][l:l+mini_batch_size] = output.index_select(1,second_column).resize(mini_batch_size)

/Users/churchhyll/opt/anaconda3/lib/python3.8/site-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


KeyboardInterrupt: 

In [ ]:
def showTensor(aTensor):
    plt.figure(1)
    figure, axes = plt.subplots()
    color_map = plt.cm.get_cmap('RdYlBu')
    reversed_color_map = color_map.reversed()
    
    im = imshow(aTensor.numpy(), origin='lower', extent=[0,1,0,1], cmap=reversed_color_map)
    
    #cset = contour(aTensor,arange(0.5,1.5,1),linewidths=1.0,
    #                   origin='lower', extent=[0,1,0,1])
    #clabel(cset,inline=True,fmt='%1.1f',fontsize=10)
    
    # Radius of our circle
    axes = plt.gca()
    R = 1 / math.sqrt(2 * math.pi)
    Drawing_uncolored_circle = plt.Circle((0.5, 0.5 ), R, fill=False, linestyle = '--', label='Target circle')
    axes.set_aspect(1)
    axes.add_artist(Drawing_uncolored_circle)
    
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Plot of the results given by the trained model')
    plt.legend()
    plt.legend(handles=[Drawing_uncolored_circle], loc='upper right')
    
    colorbar(im)
    plt.savefig('Final.jpg')


showTensor(Z);

plt.figure(2)
Label=train_classes.view(-1).float() # 1- to display the other label
x1=(train_input.narrow(1,0,1).view(-1)*Label);
y1=(train_input.narrow(1,1,1).view(-1)*Label);
x0=(train_input.narrow(1,0,1).view(-1)*(1-Label));
y0=(train_input.narrow(1,1,1).view(-1)*(1-Label));
plt.figure(num=None, figsize=(8, 4), dpi=80, facecolor='w', edgecolor='k')

plt.gca().set_aspect('equal', adjustable='box')
axes = plt.gca();
Lab0, = plt.plot(x1, y1, 'C3o', label='Label 0');
Lab1, = plt.plot(x0, y0, 'C0o', label='Label 1');
plt.xlabel('x')
plt.ylabel('y')

R = 1 / math.sqrt(2 * math.pi)
Drawing_uncolored_circle = plt.Circle((0.5, 0.5 ), R, fill=False, linestyle = '--', label='Target circle')
axes.set_aspect(1)
axes.add_artist(Drawing_uncolored_circle)

plt.title('Plot of train set and the target cicle')
plt.legend()
plt.legend(handles=[Lab1, Lab0, Drawing_uncolored_circle], loc='upper right')
axes.set_xlim([0,1]);
axes.set_ylim([0,1]);
plt.savefig('Train.jpg')